In [2]:
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix
from scipy.stats.mstats import gmean
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

fpath = "/Users/ys8mz/Box Sync/Predictive Models of College Completion (VCCS)/intermediate_files"
results_dir = "C:\\Users\\ys8mz\\Box Sync\\Predictive Models of College Completion (VCCS)\\evaluation_results\\truncated_predictors\\"

In [45]:
df = pd.read_stata(fpath + "/full_data_truncated_survival.dta")
for p in ['num_terms', 'times', 'event']:
    df.loc[:,p] = df[p].astype(int)

In [46]:
train_df = df[df.valid == 0].drop(['valid'], axis=1)
test_df = df[df.valid == 1].drop(['valid'], axis=1)
print(train_df.shape,test_df.shape)

(298139, 334) (33115, 334)


In [50]:
# degree completion rate of training sample
print(sum(train_df.event)/train_df.shape[0])
# degree completion rate of validation sample
print(sum(test_df.event)/test_df.shape[0])

0.342028382734228
0.3414361637878971


In [51]:
impute_list_1 = set(["prop_comp_pre","cum_gpa_pre"])
impute_list_2 = set([t1+"_"+t2+str(t3) for t1 in ["term_gpa", "prop_comp", "lvl2_prop_comp", "dev_prop_comp"] for t2 in ["fa", "sp", "su"] for t3 in range(1,7,1)])
impute_list_3 = set(["cum_gpa", "lvl2_prop_comp", "dev_prop_comp", "prop_comp", "prop_comp_sd", "withdrawn_prop_comp_sd"])
impute_list_4 = set(["admrate", "gradrate", "satvr25", "satvr75", "satmt25", "satmt75", "satwr25", "satwr75"])

In [52]:
def impute(train, test):
    for p in impute_list_1:
        avg_p = np.nanmean(train[train.enrolled_pre == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_3:
        avg_p = np.nanmean(train[p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_2:
        suffix = p[-3:]
        avg_p = np.nanmean(train[train["enrolled_" + suffix] == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    for p in impute_list_4:
        avg_p = np.nanmean(train[train["enrolled_nsc"] == 1][p])
        train.loc[:,p] = train.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
        test.loc[:,p] = test.loc[:,p].apply(lambda x: avg_p if pd.isnull(x) else x)
    return train, test   

In [57]:
%time train_df_new, test_df_new = impute(train_df, test_df)

Wall time: 1min 36s


In [58]:
y_test = test_df_new.event.copy()
test_df_new.loc[:,'event'] = 0

In [40]:
# Fix the CoxPH model
cph = CoxPHFitter(penalizer=0.01) # No penalizer will result in LinAlgError (probably due to multicollinearity)
%time cph.fit(train_df_new, duration_col='times', event_col='event', strata='num_terms')

C:\Users\ys8mz\AppData\Roaming\Python\Python35\site-packages\lifelines\utils\__init__.py:1086: ConvergenceWarning: Column(s) ['enrl_intensity_nsc_sp6', 'enrl_intensity_nsc_su6'] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


Wall time: 3min 44s


<lifelines.CoxPHFitter: fitted with 298139 total observations, 196167 right-censored observations>

In [10]:
with open(results_dir + 'cox.pickle', 'wb') as f:
    pickle.dump(cph, f) # saving my trained cph model as my.pickle, which is reusable

In [65]:
# Predict graduation probability (derived from survival probability by the end of Year 6)
test_pred_raw = cph.predict_survival_function(test_df_new).T.sort_index()
print(test_pred_raw.shape)
y_test_pred = [1 - test_pred_raw.iloc[i,:].loc[18-e] for i,e in enumerate(test_df_new.num_terms)]

(33116, 17)


In [66]:
# C-statistics
print("Cox Proportional Hazard Model:")
print("AUC = {}".format(round(roc_auc_score(y_test, y_test_pred),4)))

Cox Proportional Hazard Model:
AUC = 0.881


In [67]:
pickle.dump(list(y_test_pred), open(fpath + "/y_test_pred_cox.p", "wb"))

In [80]:
def find_optimal_threshold(p,r,t):
    to_drop = np.union1d(np.where(pd.isnull(p[:-1]) == True)[0], np.where(pd.isnull(r[:-1]) == True)[0])
    to_drop = np.union1d(to_drop, np.where(pd.isnull(t) == True)[0])
    to_keep = np.setdiff1d(np.array(list(range(len(p)-1))), to_drop)
    p,r,t = p[to_keep],r[to_keep],t[to_keep]
    f1 = 2*p*r/(p+r)
    best_t = t[np.argmax(f1)]
    best_t
    return best_t

In [101]:
def cross_validation_cox(train):
    threshold_list = []
    auc_list = []
    k_fold =  StratifiedKFold(n_splits = 10, random_state = 12345, shuffle=True)
    for train_indices, test_indices in k_fold.split(train, train.event):
        train_part = train.iloc[train_indices,:]
        test_part = train.iloc[test_indices,:]
        train_part_new, test_part_new = impute(train_part, test_part)
        test_part_new = test_part_new.sort_index()
        y_2 = test_part_new.event.copy()
        test_part_new.loc[:,'event'] = 0
        model = CoxPHFitter(penalizer=0.01)
        model.fit(train_part_new, duration_col='times', event_col='event', strata='num_terms')
        y_2_pred_raw = model.predict_survival_function(test_part_new).T.sort_index()
        y_2_pred = [1 - y_2_pred_raw.iloc[i,:].loc[18-e] for i,e in enumerate(test_part_new.num_terms)]
        p,r,t = precision_recall_curve(y_2, y_2_pred)
        auc = roc_auc_score(y_2, y_2_pred)
        threshold_list.append(find_optimal_threshold(p,r,t))
        auc_list.append(auc)
    print(threshold_list)
    print(np.mean(auc_list), np.std(auc_list, ddof=1))
    return gmean(threshold_list)   

In [82]:
best_threshold = cross_validation_cox(train_df)

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

...

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\ys8mz\AppData\Local\Continuum\Anaconda3\envs\Survival\lib\site-packages\pandas\core\indexin

0.8773601576272073
[0.3821016230050689, 0.38078621350317876, 0.37015606607343265, 0.35070980196827894, 0.3515859437367237, 0.3663877206878521, 0.3849610414792659, 0.3821238362075996, 0.35497146109742495, 0.3510465111829887]
0.8783822536599626 0.0013760829246604336


In [2]:
print(best_threshold)

0.36722685193132715


In [14]:
def create_confusion_matrix(y_test_pred, threshold, fname):
    cm_arr = confusion_matrix(y_test, np.where(y_test_pred > threshold, 1, 0))
    cm_df = pd.DataFrame(cm_arr, columns=['Pred_0','Pred_1'], index=['Real_0', 'Real_1'])
    cm_df.loc[:,''] = cm_df.sum(axis=1)
    cm_df.loc['',:] = cm_df.sum(axis=0)
    print(cm_df)
    print("")
    p1 = cm_df.iloc[1,1]/cm_df.iloc[2,1]
    r1 = cm_df.iloc[1,1]/cm_df.iloc[1,2]
    p0 = cm_df.iloc[0,0]/cm_df.iloc[2,0]
    r0 = cm_df.iloc[0,0]/cm_df.iloc[0,2]    
    print("F1 score = {}".format(round(2*p1*r1/(p1+r1),4)))    
    cm_df.to_csv(results_dir + fname + ".csv")
    return p1,r1,p0,r0,round(2*p1*r1/(p1+r1),4)

In [16]:
print("F1 threshold = {}:\n".format(str(round(best_threshold,3))))
pr_cox = create_confusion_matrix(np.array(y_test_pred), best_threshold, "Cox_cm1")

F1 threshold = 0.367:

         Pred_0   Pred_1         
Real_0  18347.0   3461.0  21808.0
Real_1   2619.0   8688.0  11307.0
        20966.0  12149.0  33115.0

F1 score = 0.7408


In [17]:
num_of_0 = int(round((1-np.mean(train_df.event))*len(y_test)))
y_test_pred_binary = np.ones(len(y_test))
y_test_pred_binary[np.argsort(y_test_pred)[:num_of_0]] = 0
alternative_threshold = y_test_pred[np.argsort(y_test_pred)[num_of_0]]
print("Alternative threshold = {}:\n".format(str(round(alternative_threshold,3))))
pr2_cox = create_confusion_matrix(y_test_pred_binary, best_threshold, "Cox_cm2")

Alternative threshold = 0.398:

         Pred_0   Pred_1         
Real_0  18814.0   2994.0  21808.0
Real_1   2975.0   8332.0  11307.0
        21789.0  11326.0  33115.0

F1 score = 0.7363


In [18]:
precision_recall_df = pd.DataFrame([(best_threshold,)+pr_cox,(alternative_threshold,)+pr2_cox]).round(4)
precision_recall_df.index = ['F1','Same_Graduation_Rate']
precision_recall_df.columns = ['threshold','precision_1','recall_1','precision_0','recall_0','f1_score']
precision_recall_df.to_csv(results_dir + "Cox_precision_recall.csv", index=True)

In [21]:
precision_recall_df

,threshold,precision_1,recall_1,precision_0,recall_0,f1_score
F1,0.3672,0.7151,0.7684,0.8751,0.8413,0.7408
Same_Graduation_Rate,0.3981,0.7357,0.7369,0.8635,0.8627,0.7363
